In [1]:
# import packages
import pandas as pd
import json
#from flat_table import normalize
import ast
import re
import math

In [2]:
# load the package
df = pd.read_csv('/project/df_US_2020-12-01_2020-12-15.csv')
df.head()

,lens_id,jurisdiction,kind,date_published,doc_key,docdb_id,lang,biblio,families,legal_status,abstract,claims,description,publication_type
0,000-121-388-115-77X,US,B2,2019-12-03,US_10493230_B2_20191203,522839733.0,en,{'publication_reference': {'jurisdiction': 'US...,{'simple_family': {'members': [{'document_id':...,"{'granted': True, 'grant_date': '2019-12-03', ...",[{'text': 'An intermittent urinary catheter 1 ...,[{'claims': [{'claim_text': ['1. An intermitte...,{'text': 'BACKGROUND Urinary catheter assembli...,GRANTED_PATENT
1,000-261-209-357-517,US,B2,2019-12-03,US_10496787_B2_20191203,522835547.0,en,{'publication_reference': {'jurisdiction': 'US...,{'simple_family': {'members': [{'document_id':...,"{'granted': True, 'grant_date': '2019-12-03', ...",[{'text': 'A method of monitoring usage of a c...,[{'claims': [{'claim_text': ['1. A method of m...,"{'text': ""BACKGROUND Technical Field The prese...",GRANTED_PATENT
2,000-432-924-755-657,US,B2,2019-12-10,US_10505349_B2_20191210,523241938.0,en,{'publication_reference': {'jurisdiction': 'US...,{'simple_family': {'members': [{'document_id':...,"{'granted': True, 'grant_date': '2019-12-10', ...",[{'text': 'A device or component thereof is pr...,[{'claims': [{'claim_text': ['1. A device for ...,{'text': 'The present invention relates to a d...,GRANTED_PATENT
3,000-634-853-506-789,US,S,2019-12-03,US_D0869019_S_20191203,522842605.0,en,{'publication_reference': {'jurisdiction': 'US...,{'simple_family': {'members': [{'document_id':...,"{'granted': True, 'grant_date': '2019-12-03', ...",NaN,NaN,NaN,DESIGN_RIGHT
4,001-304-722-496-247,US,S,2019-12-10,US_D0869549_S_20191210,523244272.0,en,{'publication_reference': {'jurisdiction': 'US...,{'simple_family': {'members': [{'document_id':...,"{'granted': True, 'grant_date': '2019-12-10', ...",NaN,NaN,NaN,DESIGN_RIGHT


In [3]:
# convert to json
def convertJSON(series):
    series = ast.literal_eval(series)
    if type(series) is dict:
        return series
    elif type(series) is list:
        series = json.dumps(series)
        series = ast.literal_eval(series)
        return series
    else:
        None

In [4]:
# covert to json
df['biblio'] = df['biblio'].apply(convertJSON)
df['abstract'] = df['abstract'].fillna("[{'text': '', 'lang': ''}]").apply(convertJSON)
df['claims'] = df['claims'].fillna("[{'claims':[], 'lang':''}]").apply(convertJSON)

In [5]:
# drop columns
##df['families'] = df['families'].apply(convertJSON)
##df['legal_status'] = df['legal_status'].apply(convertJSON)
df = df.drop(columns=['families'])
df = df.drop(columns=['legal_status'])

In [6]:
df.head()

,lens_id,jurisdiction,kind,date_published,doc_key,docdb_id,lang,biblio,abstract,claims,description,publication_type
0,000-121-388-115-77X,US,B2,2019-12-03,US_10493230_B2_20191203,522839733.0,en,{'publication_reference': {'jurisdiction': 'US...,[{'text': 'An intermittent urinary catheter 1 ...,[{'claims': [{'claim_text': ['1. An intermitte...,{'text': 'BACKGROUND Urinary catheter assembli...,GRANTED_PATENT
1,000-261-209-357-517,US,B2,2019-12-03,US_10496787_B2_20191203,522835547.0,en,{'publication_reference': {'jurisdiction': 'US...,[{'text': 'A method of monitoring usage of a c...,[{'claims': [{'claim_text': ['1. A method of m...,"{'text': ""BACKGROUND Technical Field The prese...",GRANTED_PATENT
2,000-432-924-755-657,US,B2,2019-12-10,US_10505349_B2_20191210,523241938.0,en,{'publication_reference': {'jurisdiction': 'US...,[{'text': 'A device or component thereof is pr...,[{'claims': [{'claim_text': ['1. A device for ...,{'text': 'The present invention relates to a d...,GRANTED_PATENT
3,000-634-853-506-789,US,S,2019-12-03,US_D0869019_S_20191203,522842605.0,en,{'publication_reference': {'jurisdiction': 'US...,"[{'text': '', 'lang': ''}]","[{'claims': [], 'lang': ''}]",NaN,DESIGN_RIGHT
4,001-304-722-496-247,US,S,2019-12-10,US_D0869549_S_20191210,523244272.0,en,{'publication_reference': {'jurisdiction': 'US...,"[{'text': '', 'lang': ''}]","[{'claims': [], 'lang': ''}]",NaN,DESIGN_RIGHT


In [7]:
# parse a valid JSON string and convert it into a Python Dictionary
df_json= json.loads(df.to_json(orient="records"))
## print(df_json)
# normalize json
normalized_df = pd.json_normalize(df_json)

In [8]:
# parse a valid JSON string and convert it into a Python Dictionary
normalized_df_json= json.loads(normalized_df.to_json(orient="records"))
##print(normalized_df_json)

In [9]:
# Normalize semi-structured JSON data into a flat table
abstract_df = pd.json_normalize(normalized_df_json, record_path=['abstract'], meta=['lens_id', 'jurisdiction', 'kind', 'date_published', 'doc_key',
       'docdb_id', 'lang', 'description',
       'publication_type', 'biblio.publication_reference.jurisdiction',
       'biblio.publication_reference.doc_number',
       'biblio.publication_reference.kind',
       'biblio.publication_reference.date',
       'biblio.application_reference.jurisdiction',
       'biblio.application_reference.doc_number',
       'biblio.application_reference.kind',
       'biblio.application_reference.date', 'biblio.priority_claims.claims',
       'biblio.invention_title', 'biblio.parties.applicants',
       'biblio.parties.inventors', 'biblio.parties.agents',
       'biblio.parties.owners_all',
       'biblio.classifications_ipcr.classifications',
       'biblio.classifications_cpc.classifications',
       'biblio.references_cited.citations',
       'biblio.references_cited.patent_count', 'biblio.cited_by.patents',
       'biblio.cited_by.patent_count',
       'biblio.references_cited.npl_count',
       'biblio.references_cited.npl_resolved_count',
       'biblio.classifications_national.classifications'], record_prefix='abstract.')
# abstract_df.head()

In [10]:
# Normalize semi-structured JSON data into a flat table
claims_df = pd.json_normalize(normalized_df_json, record_path=['claims'], meta=['lens_id', 'jurisdiction', 'kind', 'date_published', 'doc_key',
       'docdb_id', 'lang', 'description',
       'publication_type', 'biblio.publication_reference.jurisdiction',
       'biblio.publication_reference.doc_number',
       'biblio.publication_reference.kind',
       'biblio.publication_reference.date',
       'biblio.application_reference.jurisdiction',
       'biblio.application_reference.doc_number',
       'biblio.application_reference.kind',
       'biblio.application_reference.date', 'biblio.priority_claims.claims',
       'biblio.invention_title', 'biblio.parties.applicants',
       'biblio.parties.inventors', 'biblio.parties.agents',
       'biblio.parties.owners_all',
       'biblio.classifications_ipcr.classifications',
       'biblio.classifications_cpc.classifications',
       'biblio.references_cited.citations',
       'biblio.references_cited.patent_count', 'biblio.cited_by.patents',
       'biblio.cited_by.patent_count',
       'biblio.references_cited.npl_count',
       'biblio.references_cited.npl_resolved_count',
       'biblio.classifications_national.classifications'], record_prefix='claims.')
claims_df.head()

,claims.claims,claims.lang,lens_id,jurisdiction,kind,date_published,doc_key,docdb_id,lang,description,...,biblio.parties.owners_all,biblio.classifications_ipcr.classifications,biblio.classifications_cpc.classifications,biblio.references_cited.citations,biblio.references_cited.patent_count,biblio.cited_by.patents,biblio.cited_by.patent_count,biblio.references_cited.npl_count,biblio.references_cited.npl_resolved_count,biblio.classifications_national.classifications
0,[{'claim_text': ['1. An intermittent urinary c...,en,000-121-388-115-77X,US,B2,2019-12-03,US_10493230_B2_20191203,5.2284e+08,en,{'text': 'BACKGROUND Urinary catheter assembli...,...,"[{'recorded_date': '2017-08-31', 'execution_da...",[{'symbol': 'A61M25/00'}],"[{'symbol': 'A61M25/0017'}, {'symbol': 'A61M25...","[{'sequence': 1, 'patcit': {'num': 1, 'documen...",15,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",2,None,None,None
1,[{'claim_text': ['1. A method of monitoring us...,en,000-261-209-357-517,US,B2,2019-12-03,US_10496787_B2_20191203,5.22836e+08,en,"{'text': ""BACKGROUND Technical Field The prese...",...,"[{'recorded_date': '2014-07-02', 'execution_da...","[{'symbol': 'B64F5/60'}, {'symbol': 'G01L5/00'...","[{'symbol': 'B64F5/60'}, {'symbol': 'B64F5/60'...","[{'sequence': 1, 'patcit': {'num': 1, 'documen...",70,None,None,18,2,None
2,"[{'claim_text': ['1. A device for generation, ...",en,000-432-924-755-657,US,B2,2019-12-10,US_10505349_B2_20191210,5.23242e+08,en,{'text': 'The present invention relates to a d...,...,"[{'recorded_date': '2019-06-25', 'execution_da...","[{'symbol': 'H02B13/045'}, {'symbol': 'C08F210...","[{'symbol': 'C08F210/12'}, {'symbol': 'C08F210...","[{'sequence': 1, 'patcit': {'num': 1, 'documen...",25,None,None,3,None,None
3,[],,000-634-853-506-789,US,S,2019-12-03,US_D0869019_S_20191203,5.22843e+08,en,None,...,"[{'recorded_date': '2018-01-18', 'execution_da...",None,None,"[{'sequence': 1, 'patcit': {'num': 1, 'documen...",14,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",1,4,None,"[{'symbol': '2606'}, {'symbol': 'D26/28'}]"
4,[],,001-304-722-496-247,US,S,2019-12-10,US_D0869549_S_20191210,5.23244e+08,en,None,...,"[{'recorded_date': '2018-12-11', 'execution_da...",None,None,"[{'sequence': 1, 'patcit': {'num': 1, 'documen...",47,None,None,7,None,"[{'symbol': '1603'}, {'symbol': 'D18/43'}]"


In [11]:
# covert list to string
def listToString(series):
    claim_text = ""
    if(len(series)==0):
        return None
    for jsonObject in series:
        for x in jsonObject['claim_text']:
            claim_text += x
        claim_text += "\n"
    return claim_text

In [12]:
# merge data
merged_df = abstract_df.merge(claims_df[['claims.claims', 'lens_id', 'doc_key','docdb_id']], 
                              how='inner', on=['lens_id', 'doc_key','docdb_id'])
# merged_df = merged_df.explode('claims.claim_text')
merged_df['claims.claims']= merged_df['claims.claims'].apply(listToString)

In [13]:
merged_df.head()

,abstract.text,abstract.lang,lens_id,jurisdiction,kind,date_published,doc_key,docdb_id,lang,description,...,biblio.classifications_ipcr.classifications,biblio.classifications_cpc.classifications,biblio.references_cited.citations,biblio.references_cited.patent_count,biblio.cited_by.patents,biblio.cited_by.patent_count,biblio.references_cited.npl_count,biblio.references_cited.npl_resolved_count,biblio.classifications_national.classifications,claims.claims
0,An intermittent urinary catheter 1 with a gene...,en,000-121-388-115-77X,US,B2,2019-12-03,US_10493230_B2_20191203,5.2284e+08,en,{'text': 'BACKGROUND Urinary catheter assembli...,...,[{'symbol': 'A61M25/00'}],"[{'symbol': 'A61M25/0017'}, {'symbol': 'A61M25...","[{'sequence': 1, 'patcit': {'num': 1, 'documen...",15,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",2,None,None,None,1. An intermittent urinary catheter comprising...
1,A method of monitoring usage of a component of...,en,000-261-209-357-517,US,B2,2019-12-03,US_10496787_B2_20191203,5.22836e+08,en,"{'text': ""BACKGROUND Technical Field The prese...",...,"[{'symbol': 'B64F5/60'}, {'symbol': 'G01L5/00'...","[{'symbol': 'B64F5/60'}, {'symbol': 'B64F5/60'...","[{'sequence': 1, 'patcit': {'num': 1, 'documen...",70,None,None,18,2,None,1. A method of monitoring usage of a component...
2,A device or component thereof is provided for ...,en,000-432-924-755-657,US,B2,2019-12-10,US_10505349_B2_20191210,5.23242e+08,en,{'text': 'The present invention relates to a d...,...,"[{'symbol': 'H02B13/045'}, {'symbol': 'C08F210...","[{'symbol': 'C08F210/12'}, {'symbol': 'C08F210...","[{'sequence': 1, 'patcit': {'num': 1, 'documen...",25,None,None,3,None,None,"1. A device for generation, transmission, dist..."
3,,,000-634-853-506-789,US,S,2019-12-03,US_D0869019_S_20191203,5.22843e+08,en,None,...,None,None,"[{'sequence': 1, 'patcit': {'num': 1, 'documen...",14,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",1,4,None,"[{'symbol': '2606'}, {'symbol': 'D26/28'}]",None
4,,,001-304-722-496-247,US,S,2019-12-10,US_D0869549_S_20191210,5.23244e+08,en,None,...,None,None,"[{'sequence': 1, 'patcit': {'num': 1, 'documen...",47,None,None,7,None,"[{'symbol': '1603'}, {'symbol': 'D18/43'}]",None


In [14]:
# convert from list to string
def listToString(series):
    text = ""
    # print(type(series))
    if(len(series)==0):
        return None
    for jsonObject in series:
        keys = list(set(jsonObject.keys()))
        if(len(keys) == 1):
            key =keys[0]
            value = jsonObject[key]
            if(type(value) is list):
                for x in jsonObject[key]:
                    text += x
                text += "\n"
            elif(type(value) is str):
                text += value + ","
    return text

In [15]:
# merge data
merged_df = abstract_df.merge(claims_df[['claims.claims', 'lens_id', 'doc_key','docdb_id']], 
                              how='inner', on=['lens_id', 'doc_key','docdb_id'])
# merged_df = merged_df.explode('claims.claim_text')
merged_df['claims.claims']= merged_df['claims.claims'].apply(listToString)
merged_df['biblio.classifications_cpc.classifications'] = merged_df['biblio.classifications_cpc.classifications'].fillna("").apply(listToString)

In [16]:
merged_df.head()

,abstract.text,abstract.lang,lens_id,jurisdiction,kind,date_published,doc_key,docdb_id,lang,description,...,biblio.classifications_ipcr.classifications,biblio.classifications_cpc.classifications,biblio.references_cited.citations,biblio.references_cited.patent_count,biblio.cited_by.patents,biblio.cited_by.patent_count,biblio.references_cited.npl_count,biblio.references_cited.npl_resolved_count,biblio.classifications_national.classifications,claims.claims
0,An intermittent urinary catheter 1 with a gene...,en,000-121-388-115-77X,US,B2,2019-12-03,US_10493230_B2_20191203,5.2284e+08,en,{'text': 'BACKGROUND Urinary catheter assembli...,...,[{'symbol': 'A61M25/00'}],"A61M25/0017,A61M25/0021,A61M25/0017,A61M25/002...","[{'sequence': 1, 'patcit': {'num': 1, 'documen...",15,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",2,None,None,None,1. An intermittent urinary catheter comprising...
1,A method of monitoring usage of a component of...,en,000-261-209-357-517,US,B2,2019-12-03,US_10496787_B2_20191203,5.22836e+08,en,"{'text': ""BACKGROUND Technical Field The prese...",...,"[{'symbol': 'B64F5/60'}, {'symbol': 'G01L5/00'...","B64F5/60,B64F5/60,G06F30/15,G01L5/00,G05B19/40...","[{'sequence': 1, 'patcit': {'num': 1, 'documen...",70,None,None,18,2,None,1. A method of monitoring usage of a component...
2,A device or component thereof is provided for ...,en,000-432-924-755-657,US,B2,2019-12-10,US_10505349_B2_20191210,5.23242e+08,en,{'text': 'The present invention relates to a d...,...,"[{'symbol': 'H02B13/045'}, {'symbol': 'C08F210...","C08F210/12,C08F210/12,C09K3/10,C09K3/10,H02B13...","[{'sequence': 1, 'patcit': {'num': 1, 'documen...",25,None,None,3,None,None,"1. A device for generation, transmission, dist..."
3,,,000-634-853-506-789,US,S,2019-12-03,US_D0869019_S_20191203,5.22843e+08,en,None,...,None,None,"[{'sequence': 1, 'patcit': {'num': 1, 'documen...",14,"[{'document_id': {'jurisdiction': 'US', 'doc_n...",1,4,None,"[{'symbol': '2606'}, {'symbol': 'D26/28'}]",None
4,,,001-304-722-496-247,US,S,2019-12-10,US_D0869549_S_20191210,5.23244e+08,en,None,...,None,None,"[{'sequence': 1, 'patcit': {'num': 1, 'documen...",47,None,None,7,None,"[{'symbol': '1603'}, {'symbol': 'D18/43'}]",None


In [17]:
# drop columns
merged_df = merged_df.drop(columns=['abstract.lang','doc_key','docdb_id','lang','description',
                                   'biblio.references_cited.citations','biblio.references_cited.patent_count',
                                   'biblio.cited_by.patents','biblio.cited_by.patent_count',
                                   'biblio.references_cited.npl_count','biblio.references_cited.npl_resolved_count',
                                   'biblio.classifications_national.classifications','biblio.publication_reference.jurisdiction',
                                   'biblio.publication_reference.doc_number','biblio.publication_reference.kind',
                                   'biblio.publication_reference.date','biblio.application_reference.date',
                                   'biblio.priority_claims.claims','biblio.parties.applicants','biblio.parties.inventors',
                                   'biblio.parties.agents','biblio.parties.owners_all','biblio.application_reference.jurisdiction','biblio.application_reference.doc_number',
                                   'biblio.application_reference.kind','biblio.classifications_ipcr.classifications'])
merged_df.head()


,abstract.text,lens_id,jurisdiction,kind,date_published,publication_type,biblio.invention_title,biblio.classifications_cpc.classifications,claims.claims
0,An intermittent urinary catheter 1 with a gene...,000-121-388-115-77X,US,B2,2019-12-03,GRANTED_PATENT,"{'text': 'Intermittent urinary catheter', 'lan...","A61M25/0017,A61M25/0021,A61M25/0017,A61M25/002...",1. An intermittent urinary catheter comprising...
1,A method of monitoring usage of a component of...,000-261-209-357-517,US,B2,2019-12-03,GRANTED_PATENT,{'text': 'System and method of rotorcraft usag...,"B64F5/60,B64F5/60,G06F30/15,G01L5/00,G05B19/40...",1. A method of monitoring usage of a component...
2,A device or component thereof is provided for ...,000-432-924-755-657,US,B2,2019-12-10,GRANTED_PATENT,"{'text': 'Device for the generation, transmiss...","C08F210/12,C08F210/12,C09K3/10,C09K3/10,H02B13...","1. A device for generation, transmission, dist..."
3,,000-634-853-506-789,US,S,2019-12-03,DESIGN_RIGHT,"{'text': 'Bike lamp', 'lang': 'en'}",None,None
4,,001-304-722-496-247,US,S,2019-12-10,DESIGN_RIGHT,{'text': 'Bearing for process cartridge for im...,None,None


In [18]:
# Parse JSON 
merged_df_json= json.loads(merged_df.to_json(orient="records"))

In [19]:
# normalize json
final_df = pd.json_normalize(merged_df_json)
#final_df = final_df.drop(columns=['biblio.invention_title.lang'])

In [20]:
final_df.head()

,abstract.text,lens_id,jurisdiction,kind,date_published,publication_type,biblio.classifications_cpc.classifications,claims.claims,biblio.invention_title.text,biblio.invention_title.lang
0,An intermittent urinary catheter 1 with a gene...,000-121-388-115-77X,US,B2,2019-12-03,GRANTED_PATENT,"A61M25/0017,A61M25/0021,A61M25/0017,A61M25/002...",1. An intermittent urinary catheter comprising...,Intermittent urinary catheter,en
1,A method of monitoring usage of a component of...,000-261-209-357-517,US,B2,2019-12-03,GRANTED_PATENT,"B64F5/60,B64F5/60,G06F30/15,G01L5/00,G05B19/40...",1. A method of monitoring usage of a component...,System and method of rotorcraft usage monitoring,en
2,A device or component thereof is provided for ...,000-432-924-755-657,US,B2,2019-12-10,GRANTED_PATENT,"C08F210/12,C08F210/12,C09K3/10,C09K3/10,H02B13...","1. A device for generation, transmission, dist...","Device for the generation, transmission, distr...",en
3,,000-634-853-506-789,US,S,2019-12-03,DESIGN_RIGHT,None,None,Bike lamp,en
4,,001-304-722-496-247,US,S,2019-12-10,DESIGN_RIGHT,None,None,Bearing for process cartridge for image formin...,en


In [21]:
final_df.to_csv('clean_data.csv')